# Q1.Data Processing
1. 결측치 대체 :   
   머신러닝/딥러닝 기반의 결측치 대체 모델을 활용하여 결측치를 효과적으로 보간한 후, 분류 모델을 개발.
   
> 결측치 대체 모델 활용 -> 분류모델 생성   
> 당뇨 및 고혈압 유병에 대한 각 분류 모델 생성 시, 클래스 불균형문제 고려. -> 불균형 시 해결 전략 도입하여 성능 개선(challenge)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, classification_report

import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings

warnings.filterwarnings("ignore")

### 데이터 확인
- 건강검진 및 생체 정보 데이터셋   
- 31 column   
- 시계열 데이터X, 변수별 연관성은 있을 것 같음.

In [ ]:
# 데이터 확인
import pandas as pd
data = pd.read_csv("./Data_01.csv")

# print(data.head())  # 데이터 일부 출력
# print(data.info())  # 데이터 정보 출력
# print(data.describe())  # 데이터 통계 정보 출력

---
### **Flow**   
> step01 : 변수별 결측 수치(ratio) 확인   
> - 완전히 결측된 colums data 3col 발견 --> 제거   
> - target colums : y에 해당하는 컬럼. --> 해당 칼럼에도 24개 결측치 있음. 우선 해당 작업에서는 drop
   #   
> setp02 : 모델 선정   
> - Autoencoder   
> - GAIN (Generative Adversarial Imputation Network)   
   #
> setp03 : Baseline으로 MICE, KNN 등 비교    
    

---
> step01   
to-do. 결측치 컬럼 제거 후 다시 확인

In [3]:
# 완전한 결측치 제거
drop_cols = ['HE_Ucot', 'HE_FVC', 'HE_Frtn']
df = data.drop(columns=drop_cols)

In [6]:
missing_count = df.isnull().sum()     # 칼럼별 결측치 개수
missing_ratio = df.isnull().mean() * 100

# 보기 좋게 하나의 데이터프레임으로 정리
missing_df = pd.DataFrame({
    'Missing Count': missing_count,
    'Missing Ratio (%)': missing_ratio
})

# 결측치가 있는 변수만 필터링 
missing_df = missing_df[missing_df['Missing Count'] > 0]

# 결측치 많은 순으로 정렬
missing_df = missing_df.sort_values(by='Missing Count', ascending=False)

print(missing_df)

           Missing Count  Missing Ratio (%)
HE_Bplt            11882          59.412971
HE_USG              6941          34.706735
HE_Uph              6941          34.706735
HE_WBC              6759          33.796690
HE_HbA1c            6189          30.946547
HE_LDL              3122          15.610781
HE_TG                333           1.665083
HE_glu               330           1.650083
HE_ALT               262           1.310066
HE_AST               227           1.135057
HE_HB                219           1.095055
HE_HCT               215           1.075054
HE_RBC               213           1.065053
HE_BUN               213           1.065053
HE_CHOL              212           1.060053
HE_HDL               210           1.050053
HE_CREA              187           0.935047
HE_PLS               169           0.845042
Target_DM             24           0.120006
Target_HT             24           0.120006
HE_WC                  4           0.020001


In [8]:
# 타겟 변수 분리
target_cols = ['Target_DM', 'Target_HT']
# y_DM = df['Target_DM'].astype(int)  # 당뇨 질병 유병 여부
# y_HT = df['Target_HT'].astype(int)  # 고혈압 질병 유병 여부
targets = df[target_cols]
features = df.drop(columns=target_cols)

# features로만 결측치 대체해보기
features

,SEX,AGE,HE_WT,HE_HT,HE_WC,HE_BMI,HE_SBP,HE_DBP,HE_PLS,HE_Uph,...,HE_AST,HE_ALT,HE_CHOL,HE_TG,HE_HDL,HE_LDL,HE_glu,HE_HbA1c,HE_BUN,HE_CREA
0,2,50,64.24,157.8,96.8,25.798326,152.0,86.0,17.0,6.0,...,27.0,27.0,229.0,113.0,47.0,159.4,261.0,10.3,14.0,0.8
1,1,32,78.24,169.5,92.2,27.232621,111.0,71.0,20.0,5.0,...,30.0,27.0,274.0,173.0,43.0,196.4,69.0,4.0,19.0,1.1
2,2,31,53.60,150.7,69.3,23.601428,118.0,68.0,21.0,5.0,...,15.0,15.0,163.0,82.0,64.0,82.6,95.0,5.0,10.0,0.8
3,2,32,59.56,154.3,78.8,25.016286,116.0,83.0,18.0,5.0,...,20.0,12.0,145.0,64.0,70.0,62.2,94.0,4.5,11.0,0.8
4,2,67,57.70,146.2,89.2,26.994859,155.0,95.0,17.0,6.5,...,28.0,21.0,171.0,85.0,51.0,103.0,122.0,5.5,15.0,0.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19994,1,34,53.00,161.5,72.9,20.320333,121.0,81.0,17.0,6.5,...,24.0,18.0,160.0,98.0,48.0,92.4,86.0,NaN,9.8,1.0
19995,2,29,61.10,163.5,78.5,22.856288,103.0,72.0,16.0,5.0,...,16.0,12.0,151.0,213.0,29.0,79.4,89.0,NaN,8.7,0.9
19996,2,61,65.50,151.0,90.5,28.726810,140.0,81.0,15.0,5.5,...,16.0,13.0,245.0,593.0,28.0,NaN,141.0,7.8,18.6,0.8
19997,1,57,72.60,165.0,90.5,26.666667,119.0,80.0,20.0,5.0,...,21.0,25.0,188.0,56.0,52.0,124.8,107.0,NaN,25.9,1.0


In [10]:
# 결측 마스크 저장 : features에서 결측치가 있는 위치를 mask라는 변수에 바이너리값으로 저장. --> mask값이 True인 값을 딥러닝을 통해 예측하여 구현할 것
mask = features.isnull()

# 임시 결측치 대체 (컬럼 평균으로)
features_filled = features.fillna(features.mean())

# 스케일링
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features_filled)

---
> step02 : 딥러닝 기반 모델로 결측치 대체하기   
   #
Autoencoder 기반 Imputer (딥러닝 방식)   
원리: 입력 데이터를 압축하고 복원하는 구조를 이용해 결측치를 예측    
장점: 복잡한 비선형 관계를 학습 가능

In [ ]:
from impute_model import Autoencoder
import torch.nn as nn
from utils import *

# 데이터를 텐서 형태로 수정
X_tensor = torch.tensor(features_scaled, dtype=torch.float32)

# 학습/확인/테스트 데이터셋으로 바꿈
train_loader, val_loader, test_loader = load_dataset(features_scaled)

# 모델 생성
model = Autoencoder(input_dim=X_tensor.shape[1])

# 모델 학습
model.train_model(train_loader, "autoencoder")

In [16]:
# 5. 결측치 복원
# ----------------------
model.eval()
with torch.no_grad():
    reconstructed = model(X_tensor).numpy()

X_imputed = features_scaled.copy()

In [17]:
X_imputed

array([[ 0.88536602,  0.31024556,  0.26358649, ..., -0.24889501,
        -0.07166376, -0.78372743],
       [-1.12947638, -0.83225459,  1.56243284, ..., -0.49783093,
         1.10405408,  0.86494616],
       [ 0.88536602, -0.89572683, -0.72353673, ..., -0.45831729,
        -1.01223803, -0.78372743],
       ...,
       [ 0.88536602,  1.0084401 ,  0.38048266, ..., -0.3476791 ,
         1.00999665, -0.78372743],
       [-1.12947638,  0.75455118,  1.03918331, ...,  0.        ,
         2.72654469,  0.3153883 ],
       [ 0.88536602,  0.56413449,  0.4732574 , ...,  0.        ,
         0.04590802, -1.3332853 ]])

In [18]:
# 마스크를 이용해 결측값만 대체
for i in range(X_imputed.shape[0]):
    for j in range(X_imputed.shape[1]):
        if mask.iloc[i, j]:
            X_imputed[i, j] = reconstructed[i, j]

# 역스케일링
X_final = scaler.inverse_transform(X_imputed)
df_imputed = pd.DataFrame(X_final, columns=features.columns)

# 타겟 변수 다시 붙이기
df_result = pd.concat([df_imputed, targets.reset_index(drop=True)], axis=1)

In [19]:
df_result

,SEX,AGE,HE_WT,HE_HT,HE_WC,HE_BMI,HE_SBP,HE_DBP,HE_PLS,HE_Uph,...,HE_CHOL,HE_TG,HE_HDL,HE_LDL,HE_glu,HE_HbA1c,HE_BUN,HE_CREA,Target_DM,Target_HT
0,2.0,50.0,64.24,157.8,96.8,25.798326,152.0,86.0,17.0,6.0,...,229.0,113.0,47.0,159.400000,261.0,10.300000,14.0,0.8,0.0,0.0
1,1.0,32.0,78.24,169.5,92.2,27.232621,111.0,71.0,20.0,5.0,...,274.0,173.0,43.0,196.400000,69.0,4.000000,19.0,1.1,0.0,0.0
2,2.0,31.0,53.60,150.7,69.3,23.601428,118.0,68.0,21.0,5.0,...,163.0,82.0,64.0,82.600000,95.0,5.000000,10.0,0.8,0.0,0.0
3,2.0,32.0,59.56,154.3,78.8,25.016286,116.0,83.0,18.0,5.0,...,145.0,64.0,70.0,62.200000,94.0,4.500000,11.0,0.8,0.0,0.0
4,2.0,67.0,57.70,146.2,89.2,26.994859,155.0,95.0,17.0,6.5,...,171.0,85.0,51.0,103.000000,122.0,5.500000,15.0,0.9,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19994,1.0,34.0,53.00,161.5,72.9,20.320333,121.0,81.0,17.0,6.5,...,160.0,98.0,48.0,92.400000,86.0,15.635516,9.8,1.0,0.0,0.0
19995,2.0,29.0,61.10,163.5,78.5,22.856288,103.0,72.0,16.0,5.0,...,151.0,213.0,29.0,79.400000,89.0,16.416210,8.7,0.9,0.0,0.0
19996,2.0,61.0,65.50,151.0,90.5,28.726810,140.0,81.0,15.0,5.5,...,245.0,593.0,28.0,114.930463,141.0,7.800000,18.6,0.8,1.0,1.0
19997,1.0,57.0,72.60,165.0,90.5,26.666667,119.0,80.0,20.0,5.0,...,188.0,56.0,52.0,124.800000,107.0,15.925559,25.9,1.0,0.0,0.0
